In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
import uuid
from azureml.core import Workspace
import azureml.core
import json
from azureml.core import Run, Experiment, Workspace, Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import Webservice, AciWebservice

In [2]:

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [5]:
ws = Workspace.from_config()
with open('states/model_details.json', 'r') as f:
    model_details = json.load(f)
model = Model.deserialize(workspace=ws, model_payload=model_details['model'])

In [6]:
env = Environment('imdb-training-env')
cd = CondaDependencies.create(
    pip_packages=['azureml-defaults','numpy', 'tensorflow']
)
env.python.conda_dependencies = cd
env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20211029.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "imdb-training-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-for

In [7]:
#inference_config = InferenceConfig(entry_script='ai-training/score.py', environment=env)
aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1, 
    memory_gb=1, 
    tags={"data": "imdb",  "method" : "keras"}, 
    description='Movie recommendations'
)

service_name = 'tveerdetection-svc-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws, 
                    name=service_name, 
                    models=[model], 
                    inference_config=inference_config, 
                    deployment_config=aciconfig)

UserErrorException: UserErrorException:
	Message: You must provide an InferenceConfig when deploying a model with model_framework set to Custom. Default environments are only provided for these frameworks: ['Onnx', 'ScikitLearn', 'TensorFlow', 'Multi'].
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You must provide an InferenceConfig when deploying a model with model_framework set to Custom. Default environments are only provided for these frameworks: ['Onnx', 'ScikitLearn', 'TensorFlow', 'Multi']."
    }
}

In [ ]:
service.wait_for_deployment(show_output=True)

with open("states/service_details.json", "w") as service_details:
    json.dump(service.serialize(), service_details)